In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
from pycocotools import mask as maskUtils
import torch
#from pycocotools import mask as maskUtils

def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)
    
def rle2polygon(segmentation):
    m = maskUtils.decode(segmentation) 
    m[m > 0] = 255
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
    polygons = []
    for contour in contours:
        epsilon = 0.001 * cv2.arcLength(contour, True)
        contour_approx = cv2.approxPolyDP(contour, epsilon, True)
        polygon = contour_approx.flatten().tolist()
        polygons.append(polygon)
    return polygons

def lala(a):
    x = a[::2]  # координаты x (четные индексы)
    y = a[1::2]  # координаты y (нечетные индексы)
    plt.plot(x, y)
    #plt.figure(figsize=(20, 20))
    plt.title('')
    plt.xlabel('')
    plt.ylabel('')
    plt.axis('off')
    ax = plt.gca()
    ax.set_autoscale_on(None)
    #mask = np.zeros((1000, 1000), dtype=np.uint8)
    #coords = np.array([x, y]).T
    #mask = cv2.fillPoly(mask, [coords], 1)

# Кодирование маски в формат COCO RLE
#mask_encoded = maskUtils.encode(np.asfortranarray(mask))

#print(mask_encoded)
    #plt.gca().set_xlim(0, 500)
    #plt.gca().set_ylim(0, 500)
    #plt.gca().set_xticks([0, 500])
    #plt.gca().set_yticks([0, 500])
    #plt.show()
    


2023-12-14 15:56:28.973580: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-14 15:56:28.991738: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 15:56:28.991758: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 15:56:28.991769: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 15:56:28.995443: I tensorflow/core/platform/cpu_feature_g

In [2]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

In [3]:
image = cv2.imread('snow10.jpg')
image_white = cv2.imread('white_sheet.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [4]:
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.88,
    stability_score_thresh=0.88,
    crop_n_layers=3,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=500,  # Requires open-cv to run post-processing
    output_mode='coco_rle'
)

In [5]:
masks = mask_generator.generate(image)

In [6]:
values = []
for hello in range(len(masks)):
#for hello in range(7):
    mask1 = masks[hello]['segmentation']
    polygons = rle2polygon(mask1)
    for i in polygons:
        first_two_values = polygons[0][:2]
        polygons[0].extend(first_two_values)
        #s1.append(polygons)
        print(polygons)
        values.append(polygons)
#print(values)        
#         for j in polygons:
#             normalized_x_coords = x_coords / width
#             normalized_y_coords = y_coords / height
            
    #lala(polygons[0])
    #plt.figure(figsize=(20, 20))
    #plt.show()    

[[484, 1380, 480, 1373, 476, 1369, 474, 1368, 470, 1368, 468, 1366, 468, 1359, 465, 1355, 461, 1351, 452, 1347, 439, 1347, 434, 1348, 431, 1349, 425, 1353, 414, 1358, 410, 1358, 405, 1360, 397, 1364, 392, 1369, 390, 1370, 382, 1370, 380, 1371, 376, 1376, 375, 1376, 373, 1381, 371, 1382, 369, 1384, 369, 1390, 359, 1405, 358, 1409, 358, 1415, 359, 1419, 360, 1421, 371, 1422, 376, 1421, 387, 1417, 401, 1410, 404, 1409, 414, 1409, 426, 1405, 440, 1403, 447, 1400, 450, 1397, 450, 1394, 451, 1392, 453, 1390, 455, 1389, 461, 1389, 466, 1392, 469, 1393, 472, 1393, 476, 1392, 478, 1391, 483, 1386, 484, 1384, 484, 1380]]
[[654, 873, 654, 877, 652, 880, 652, 891, 653, 894, 655, 895, 659, 895, 661, 896, 664, 899, 664, 901, 665, 903, 670, 909, 672, 912, 673, 915, 676, 916, 678, 918, 686, 922, 711, 921, 717, 919, 723, 918, 728, 918, 733, 920, 738, 920, 751, 924, 756, 927, 763, 927, 767, 925, 769, 923, 770, 920, 770, 918, 766, 912, 765, 909, 765, 906, 764, 903, 757, 893, 750, 886, 745, 882, 736, 878,

[[1201, 303, 1201, 305, 1207, 302, 1208, 307, 1220, 306, 1224, 307, 1240, 313, 1250, 318, 1265, 324, 1269, 324, 1273, 326, 1296, 331, 1305, 331, 1312, 333, 1314, 335, 1321, 338, 1325, 338, 1335, 342, 1337, 344, 1342, 346, 1360, 351, 1376, 359, 1379, 359, 1382, 360, 1384, 362, 1389, 362, 1391, 361, 1392, 359, 1392, 355, 1395, 346, 1396, 340, 1396, 329, 1394, 324, 1388, 322, 1380, 317, 1375, 310, 1371, 306, 1360, 298, 1347, 291, 1341, 289, 1324, 290, 1310, 288, 1291, 288, 1286, 287, 1280, 284, 1274, 283, 1262, 278, 1258, 277, 1252, 277, 1246, 275, 1235, 275, 1226, 279, 1220, 283, 1211, 291, 1209, 297, 1204, 302, 1201, 303]]
[[1189, 489, 1187, 487, 1177, 487, 1170, 488, 1166, 490, 1157, 490, 1153, 491, 1151, 492, 1150, 494, 1149, 497, 1149, 499, 1150, 502, 1152, 504, 1155, 506, 1161, 509, 1166, 508, 1169, 509, 1172, 511, 1176, 511, 1179, 510, 1181, 509, 1186, 504, 1188, 500, 1189, 497, 1189, 489]]
[[62, 490, 58, 486, 55, 485, 52, 485, 49, 482, 46, 481, 43, 478, 40, 474, 39, 472, 39, 467, 

[[102, 680, 100, 679, 98, 679, 96, 680, 95, 681, 94, 684, 91, 687, 84, 687, 82, 689, 72, 693, 64, 695, 60, 697, 57, 703, 56, 707, 54, 719, 54, 724, 56, 731, 57, 733, 59, 736, 63, 740, 66, 741, 80, 741, 84, 740, 89, 737, 91, 735, 93, 732, 94, 730, 95, 726, 95, 713, 97, 709, 99, 708, 102, 708, 105, 705, 108, 701, 108, 689, 106, 686, 103, 683, 102, 680]]
[[698, 918, 699, 921, 704, 922, 711, 921, 716, 919, 724, 918, 727, 918, 728, 919, 737, 920, 745, 922, 748, 923, 755, 927, 763, 927, 766, 926, 769, 923, 769, 921, 770, 919, 765, 910, 764, 905, 762, 901, 756, 892, 746, 883, 741, 880, 738, 880, 735, 878, 731, 878, 724, 880, 717, 884, 712, 888, 709, 892, 709, 895, 706, 899, 703, 906, 703, 909, 702, 912, 698, 918]]
[[730, 978, 728, 975, 724, 973, 720, 972, 714, 968, 709, 968, 704, 970, 696, 970, 694, 971, 693, 972, 692, 974, 691, 978, 691, 983, 690, 984, 690, 988, 691, 993, 692, 995, 693, 996, 695, 997, 699, 997, 704, 994, 706, 992, 706, 991, 707, 990, 709, 989, 715, 989, 720, 991, 724, 989, 7

[[1300, 978, 1297, 975, 1294, 975, 1289, 977, 1285, 979, 1282, 980, 1275, 987, 1272, 991, 1267, 1000, 1267, 1002, 1265, 1004, 1265, 1008, 1268, 1008, 1271, 1007, 1273, 1006, 1275, 1004, 1277, 1003, 1282, 1002, 1292, 997, 1300, 990, 1301, 987, 1301, 984, 1300, 978]]
[[927, 621, 924, 624, 924, 644, 926, 648, 929, 650, 930, 653, 930, 663, 924, 677, 922, 680, 914, 688, 914, 693, 916, 697, 918, 699, 935, 699, 944, 698, 953, 695, 959, 695, 967, 693, 970, 692, 974, 690, 976, 686, 977, 680, 976, 673, 973, 667, 969, 661, 966, 661, 962, 659, 962, 652, 960, 648, 958, 645, 955, 642, 954, 642, 952, 639, 947, 637, 942, 634, 934, 625, 927, 621]]
[[1232, 11, 1232, 20, 1241, 20, 1245, 17, 1247, 17, 1249, 18, 1252, 21, 1256, 30, 1259, 35, 1273, 49, 1280, 53, 1286, 59, 1289, 59, 1293, 57, 1320, 51, 1326, 51, 1333, 45, 1333, 44, 1336, 41, 1339, 37, 1341, 33, 1344, 30, 1346, 30, 1348, 28, 1348, 23, 1347, 21, 1344, 18, 1337, 15, 1317, 3, 1314, 0, 1239, 0, 1235, 5, 1232, 11]]
[[1146, 593, 1146, 596, 1156, 60

[[428, 1492, 416, 1488, 410, 1490, 401, 1495, 388, 1508, 387, 1510, 387, 1513, 380, 1521, 380, 1530, 378, 1533, 368, 1542, 366, 1546, 362, 1551, 356, 1556, 352, 1561, 349, 1568, 340, 1577, 332, 1588, 327, 1592, 321, 1600, 318, 1607, 316, 1618, 311, 1628, 308, 1642, 309, 1643, 310, 1647, 311, 1659, 314, 1664, 315, 1670, 319, 1678, 322, 1681, 325, 1682, 332, 1682, 334, 1681, 338, 1677, 347, 1673, 355, 1668, 363, 1660, 365, 1656, 370, 1652, 373, 1652, 377, 1650, 378, 1648, 391, 1636, 397, 1635, 401, 1633, 405, 1624, 407, 1616, 410, 1609, 416, 1600, 418, 1599, 421, 1595, 423, 1589, 438, 1575, 440, 1569, 441, 1561, 444, 1555, 444, 1551, 440, 1544, 440, 1536, 443, 1532, 445, 1520, 443, 1513, 440, 1509, 439, 1505, 432, 1498, 432, 1496, 428, 1492]]
[[559, 1828, 555, 1827, 549, 1828, 546, 1829, 540, 1832, 534, 1838, 528, 1840, 523, 1840, 517, 1842, 507, 1847, 504, 1849, 501, 1852, 497, 1860, 496, 1864, 495, 1870, 495, 1881, 496, 1891, 501, 1903, 501, 1908, 503, 1913, 508, 1914, 511, 1913, 514, 

...

[[768, 1000, 769, 1004, 771, 1005, 775, 1005, 777, 1004, 779, 1002, 779, 999, 777, 997, 775, 996, 770, 997, 770, 998, 768, 1000]]
[[735, 588, 733, 586, 731, 585, 727, 584, 723, 584, 715, 588, 714, 589, 713, 591, 713, 598, 714, 600, 717, 602, 721, 602, 727, 596, 729, 595, 732, 595, 734, 593, 735, 591, 735, 588]]
[[1077, 670, 1076, 672, 1076, 675, 1075, 678, 1072, 681, 1071, 683, 1071, 685, 1073, 686, 1081, 688, 1084, 686, 1088, 685, 1092, 681, 1089, 676, 1085, 676, 1080, 671, 1077, 670]]
[[822, 538, 816, 542, 811, 547, 810, 549, 811, 560, 818, 574, 819, 579, 819, 584, 817, 588, 811, 593, 809, 598, 809, 607, 810, 611, 814, 618, 814, 621, 818, 625, 824, 629, 844, 646, 863, 666, 866, 667, 869, 670, 884, 677, 897, 682, 899, 684, 909, 687, 918, 680, 918, 678, 921, 674, 921, 671, 924, 668, 927, 658, 927, 651, 926, 649, 923, 646, 921, 637, 921, 626, 923, 620, 923, 614, 921, 604, 921, 593, 918, 584, 914, 579, 910, 571, 910, 568, 907, 561, 901, 556, 899, 553, 893, 551, 886, 545, 873, 537, 868, 5

[[271, 1264, 269, 1265, 265, 1268, 255, 1272, 250, 1273, 243, 1272, 238, 1274, 236, 1276, 235, 1285, 235, 1294, 233, 1296, 232, 1296, 231, 1297, 231, 1298, 229, 1300, 227, 1304, 227, 1312, 229, 1315, 229, 1318, 230, 1321, 232, 1323, 235, 1324, 240, 1329, 247, 1332, 252, 1337, 255, 1338, 259, 1338, 263, 1336, 266, 1333, 267, 1333, 269, 1331, 269, 1328, 271, 1325, 276, 1319, 284, 1311, 285, 1307, 287, 1305, 287, 1304, 292, 1302, 292, 1298, 290, 1296, 289, 1296, 287, 1294, 286, 1288, 282, 1280, 278, 1276, 277, 1274, 276, 1269, 275, 1267, 273, 1265, 271, 1264]]
[[434, 1260, 432, 1260, 430, 1262, 428, 1269, 426, 1269, 423, 1267, 416, 1267, 413, 1264, 414, 1260, 412, 1260, 411, 1262, 403, 1262, 398, 1266, 393, 1267, 386, 1274, 385, 1277, 384, 1287, 384, 1295, 382, 1303, 380, 1306, 380, 1311, 378, 1316, 376, 1318, 376, 1321, 378, 1323, 378, 1325, 382, 1329, 384, 1330, 394, 1330, 397, 1329, 403, 1323, 405, 1319, 408, 1316, 409, 1316, 412, 1313, 417, 1302, 419, 1299, 427, 1291, 430, 1291, 434, 

[[1104, 437, 1103, 437, 1101, 435, 1099, 434, 1094, 434, 1092, 435, 1090, 437, 1086, 438, 1085, 441, 1081, 445, 1080, 448, 1076, 452, 1075, 455, 1072, 458, 1071, 461, 1066, 466, 1065, 469, 1064, 470, 1063, 470, 1061, 472, 1064, 475, 1060, 477, 1053, 478, 1046, 480, 1038, 483, 1034, 485, 1030, 489, 1028, 493, 1026, 503, 1026, 509, 1028, 514, 1033, 519, 1035, 524, 1041, 524, 1048, 521, 1050, 519, 1054, 517, 1061, 515, 1066, 515, 1074, 511, 1079, 506, 1079, 505, 1084, 500, 1088, 498, 1093, 497, 1108, 496, 1109, 494, 1110, 487, 1111, 485, 1114, 482, 1114, 479, 1112, 476, 1113, 460, 1111, 455, 1111, 450, 1107, 441, 1105, 439, 1105, 438, 1104, 437]]
[[231, 308, 231, 310, 233, 311, 244, 313, 261, 314, 267, 315, 274, 317, 280, 320, 285, 325, 286, 325, 289, 328, 295, 327, 297, 329, 298, 332, 302, 333, 302, 338, 303, 330, 301, 327, 301, 324, 298, 322, 289, 317, 280, 308, 276, 305, 268, 301, 263, 299, 259, 298, 247, 298, 244, 299, 240, 301, 236, 305, 233, 306, 231, 308]]
[[999, 1326, 997, 1327, 9

[[171, 678, 172, 684, 176, 685, 184, 683, 183, 681, 179, 679, 176, 677, 171, 678]]
[[1503, 1244, 1495, 1243, 1484, 1245, 1484, 1246, 1482, 1248, 1481, 1251, 1488, 1251, 1491, 1250, 1494, 1248, 1500, 1248, 1503, 1244]]
[[1465, 696, 1462, 696, 1461, 695, 1459, 695, 1453, 697, 1449, 699, 1446, 701, 1443, 704, 1442, 706, 1442, 713, 1444, 717, 1445, 723, 1443, 726, 1444, 738, 1444, 743, 1443, 745, 1441, 747, 1439, 748, 1434, 748, 1432, 749, 1431, 750, 1430, 752, 1431, 756, 1434, 757, 1441, 756, 1466, 757, 1469, 756, 1471, 754, 1472, 749, 1475, 744, 1477, 742, 1478, 742, 1480, 740, 1482, 737, 1483, 735, 1483, 730, 1481, 723, 1480, 721, 1476, 717, 1475, 710, 1474, 708, 1468, 699, 1465, 696]]
[[1526, 1357, 1525, 1360, 1520, 1359, 1521, 1357, 1518, 1357, 1518, 1361, 1517, 1364, 1515, 1364, 1509, 1361, 1509, 1363, 1507, 1364, 1502, 1365, 1497, 1365, 1491, 1369, 1488, 1372, 1484, 1375, 1477, 1377, 1471, 1377, 1468, 1379, 1458, 1379, 1453, 1381, 1452, 1385, 1452, 1390, 1447, 1396, 1445, 1400, 1442

[[470, 1368, 467, 1370, 466, 1372, 466, 1376, 467, 1384, 464, 1386, 463, 1389, 465, 1391, 468, 1393, 474, 1393, 477, 1392, 480, 1390, 483, 1387, 485, 1383, 485, 1380, 482, 1377, 480, 1373, 477, 1371, 477, 1370, 475, 1368, 470, 1368]]
[[1278, 813, 1267, 832, 1265, 850, 1272, 868, 1276, 889, 1283, 899, 1291, 905, 1292, 913, 1289, 918, 1277, 924, 1276, 931, 1286, 945, 1288, 952, 1299, 969, 1307, 977, 1362, 1014, 1411, 1039, 1418, 1045, 1458, 1068, 1469, 1070, 1489, 1081, 1521, 1088, 1545, 1098, 1569, 1103, 1611, 1100, 1624, 1096, 1638, 1071, 1661, 1046, 1665, 1035, 1661, 983, 1643, 933, 1635, 919, 1622, 904, 1610, 895, 1603, 877, 1591, 859, 1580, 848, 1572, 844, 1563, 834, 1559, 832, 1550, 821, 1527, 810, 1518, 809, 1508, 801, 1503, 803, 1499, 800, 1498, 797, 1500, 790, 1504, 787, 1506, 788, 1512, 785, 1507, 782, 1503, 783, 1500, 777, 1486, 765, 1468, 760, 1438, 760, 1423, 763, 1407, 761, 1369, 763, 1353, 767, 1344, 767, 1319, 779, 1308, 792, 1278, 813]]
[[1166, 1338, 1167, 1343, 1171, 13

In [7]:
data_reduced = [sublist[0] for sublist in values]

In [8]:
values = data_reduced

result = []
width, height, _ = image.shape
# Цикл для обработки каждого списка значений
for sublist in values:
    new_sublist = []
    for i, val in enumerate(sublist):
        if i % 2 == 0:  # Если индекс четный
            new_sublist.append(val / height)
        else:  # Если индекс нечетный
            new_sublist.append(val / width)
    result.append(new_sublist)

# Вывод результатов
print(result)

[[0.1890625, 0.71875, 0.1875, 0.7151041666666667, 0.1859375, 0.7130208333333333, 0.18515625, 0.7125, 0.18359375, 0.7125, 0.1828125, 0.7114583333333333, 0.1828125, 0.7078125, 0.181640625, 0.7057291666666666, 0.180078125, 0.7036458333333333, 0.1765625, 0.7015625, 0.171484375, 0.7015625, 0.16953125, 0.7020833333333333, 0.168359375, 0.7026041666666667, 0.166015625, 0.7046875, 0.16171875, 0.7072916666666667, 0.16015625, 0.7072916666666667, 0.158203125, 0.7083333333333334, 0.155078125, 0.7104166666666667, 0.153125, 0.7130208333333333, 0.15234375, 0.7135416666666666, 0.14921875, 0.7135416666666666, 0.1484375, 0.7140625, 0.146875, 0.7166666666666667, 0.146484375, 0.7166666666666667, 0.145703125, 0.7192708333333333, 0.144921875, 0.7197916666666667, 0.144140625, 0.7208333333333333, 0.144140625, 0.7239583333333334, 0.140234375, 0.7317708333333334, 0.13984375, 0.7338541666666667, 0.13984375, 0.7369791666666666, 0.140234375, 0.7390625, 0.140625, 0.7401041666666667, 0.144921875, 0.740625, 0.146875, 

In [9]:
formatted_result = str(result).replace(' [', '\n0 ').replace('[', '0 ').replace(']', '')
best_result = formatted_result.replace(',', '')
print(best_result)


0 0 0.1890625 0.71875 0.1875 0.7151041666666667 0.1859375 0.7130208333333333 0.18515625 0.7125 0.18359375 0.7125 0.1828125 0.7114583333333333 0.1828125 0.7078125 0.181640625 0.7057291666666666 0.180078125 0.7036458333333333 0.1765625 0.7015625 0.171484375 0.7015625 0.16953125 0.7020833333333333 0.168359375 0.7026041666666667 0.166015625 0.7046875 0.16171875 0.7072916666666667 0.16015625 0.7072916666666667 0.158203125 0.7083333333333334 0.155078125 0.7104166666666667 0.153125 0.7130208333333333 0.15234375 0.7135416666666666 0.14921875 0.7135416666666666 0.1484375 0.7140625 0.146875 0.7166666666666667 0.146484375 0.7166666666666667 0.145703125 0.7192708333333333 0.144921875 0.7197916666666667 0.144140625 0.7208333333333333 0.144140625 0.7239583333333334 0.140234375 0.7317708333333334 0.13984375 0.7338541666666667 0.13984375 0.7369791666666666 0.140234375 0.7390625 0.140625 0.7401041666666667 0.144921875 0.740625 0.146875 0.7401041666666667 0.151171875 0.7380208333333333 0.156640625 0.734

In [10]:
with open('snow10_1.txt', 'w') as file:
    file.write(best_result)